In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime as dt
import pickle

from scipy.spatial import Delaunay
from scipy.stats import linregress
from scipy.optimize import curve_fit
from getdist import plots, MCSamples
from iminuit import Minuit
from iminuit.cost import LeastSquares

from qubic.lib.Qgps import GPSAntenna
import qubic.lib.Calibration.Qfiber as ft
 
%matplotlib inline

In [ ]:
# If True, allow plots for debug
DEBUG = False

In [ ]:
# Distances during experiment
distance_base_antenna2 = np.array([10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1])
distance_antenna1_antenna2 = np.array([7, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 2, 3, np.sqrt(2), np.sqrt(3), 2, 1])

if distance_base_antenna2.size != distance_antenna1_antenna2.size:
    print(distance_base_antenna2.size, distance_antenna1_antenna2.size)
    raise ValueError("The two arrays must have the same size")

# Import data

In [ ]:
dict_data = pickle.load(open('GPS_noise_analysis.pkl', 'rb'))

names = np.array(['North', 'East', 'Down', 'Roll', 'Yaw'])

In [ ]:
print(dict_data.keys())
print(dict_data['rpN'].keys())
print(dict_data['rpN']['clean_data'][0].size)

# Noise Power Spectrum

## Def useful functions

In [ ]:
timestep = 0.125
print("Timestep : ", timestep, "s.")

def get_ps(array):
    """Function to compute the power spectrum of a given array.

    Parameters
    ----------
    array : array_like
        array containing the data to compute the power spectrum of.

    Returns
    -------
    power_spectrum : array_like
        array containing the power spectrum of the input array.
    freq: array_like
        array containing the frequency of the power spectrum.
    """
    N = array.size
    return np.abs(np.fft.rfft(array))**2, np.fft.rfftfreq(N, d=timestep/2)

In [ ]:
power_spectrum, freq = get_ps(dict_data['rpN']['clean_data'][0])
power_spectrum, freq = power_spectrum[1:], freq[1:]

plt.plot(freq, power_spectrum)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale('log')
plt.yscale('log')
plt.title('Noise Power Spectrum - North')

del power_spectrum, freq

In [ ]:
def noise_model(x, A_white, f_knee, slope):
    return A_white**2 * (1 + np.abs(f_knee/x)**slope)

### Fit using my Gaussian LogLikelihood function with minuit

## Fit 1/f + white with minuit

In [ ]:
def nll_exp(A, f_knee, alpha, f, ps):
    P_model = noise_model(f, A, f_knee, alpha)
    return np.sum(np.log(P_model) + ps / P_model)

def nll_gauss(A, f_knee, alpha, freq, ps, sigma):
    P_model = noise_model(freq, A, f_knee, alpha)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha):
       return nll_gauss(A, f_knee, alpha, binned_freq, binned_ps, sigma=binned_ps_error)

    m = Minuit(nll_wrapper, A=0.1, f_knee=1, alpha=1)
    m.limits['A'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        plt.plot(freq, ps, label="data")
        plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
Try to use video from scikit MOOC (module 2, Validation and learning curves) to determine the number of points I want to use (and also, which model)

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$']

# Fit for each instrumental configurations
for idata in range(len(names)):
    values, errors, chi2_dof= [], [], []
    for index in range(len(dict_data['rpN']['clean_data'])):
        if index == 0:
            plot = True
        else:
            plot = False
        val, err, red_chi2 = fit_minuit_ll(dict_data[names[idata]]['clean_data'][index], nbins=int(dict_data[names[idata]]['clean_data'][index].size/10), plot=plot, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(3, 5, figsize=(15, 10), sharey='row', sharex=True)

n_datasets = len(names)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(3):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
#    ax[1, idata].set_ylim(0, 10)

for idata in range(n_datasets):
    ax[2, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - over f + White.png')
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=5, shared_xaxes=True, shared_yaxes=True,
    subplot_titles=names,
    vertical_spacing=0.05,   # decreased spacing for larger plots
    horizontal_spacing=0.03, # decreased spacing for larger plots
    row_heights=[0.1, 0.1, 0.1],
    column_widths=[0.1, 0.1, 0.1, 0.1, 0.1]
)

n_datasets = len(names)
n_points = len(params_values_loglike[0])

for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        
        # Add error bars
        fig.add_trace(
            go.Scatter(
                x=x,
                y=values,
                error_y=dict(type='data', array=errors, visible=True),
                mode='markers',
                marker=dict(color="black"),
                name=names[idata],
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )
        
        # Add mean line
        fig.add_trace(
            go.Scatter(
                x=[0, n_points-1],
                y=[mean_val, mean_val],
                mode='lines',
                line=dict(color='red', dash='dash'),
                name=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}',
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )

# Update layout
fig.update_layout(
    height=1000,
    width=2000,
    title_text=r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$',
    showlegend=False
)

# Update y-axis labels
for iparam in range(3):
    fig.update_yaxes(title_text=label[iparam], row=iparam+1, col=1)

# Update x-axis labels
for idata in range(n_datasets):
    fig.update_xaxes(title_text='Instrumental Index', row=3, col=idata+1)

fig.show()

## Are the parameters scaled with the distance between antennas ?

In [ ]:
data_name = 'roll'
index_data = names.index(data_name)
params_value = np.array(params_values_loglike, dtype=float)[index_data]

In [ ]:

unique_distances = np.unique(distance_base_antenna2)
colors = px.colors.sample_colorscale("jet", [i/(len(unique_distances)-1) for i in range(len(unique_distances))])

# Create a subplot figure with 2 rows and 1 column, sharing the x-axis.
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
)

# Loop over unique distances to add traces for both subplots.
for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna2 == dist
    # First subplot: A_white values
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 0],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i])
        ),
        row=1, col=1
    )
    # Second subplot: f_knee values; hide legend to avoid duplicates.
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 1],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i]),
            showlegend=False
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 2],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i]),
            showlegend=False
        ),
        row=3, col=1
    )

# Update axis labels and overall layout.
fig.update_xaxes(title_text='Distance Antenna 1 to Antenna 2', row=3, col=1)
fig.update_yaxes(title_text=label[0], row=1, col=1)
fig.update_yaxes(title_text=label[1], row=2, col=1)
fig.update_yaxes(title_text=label[2], row=3, col=1)

fig.update_layout(
    title=f'Noise power spectrum parameters for different distances - {data_name} data',
    # Position the legend outside the plot on the right.
    legend=dict(
        title='Distance Base to Antenna 2',
        orientation='v',
        x=1.02,
        y=1,
        xanchor='left',
        yanchor='top'
    ),
    width=1200,
    height=800,
    margin=dict(r=150)  # Increase right margin to accommodate the legend.
)

fig.show()

In [ ]:
unique_distances = np.unique(distance_base_antenna2)
colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna2 == dist
    ax[0].scatter(distance_antenna1_antenna2[mask], params_value[mask, 0], label=f'{dist:.1f}', color=colors[i])
    ax[1].scatter(distance_antenna1_antenna2[mask], params_value[mask, 1], color=colors[i])

ax[0].set_ylabel(r'$A_{white}$')
ax[1].set_ylabel(r'$f_{knee}$')
ax[1].set_xlabel('Distance Antenna 1 to Antenna 2')

# Create a single legend outside the plot
handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 1), title='Distance Base to Antenna 2')

fig.suptitle(f'Noise power spectrum parameters for different distances - {data_name} data')
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust layout to make space for legend
plt.show()

## Fit 1/f + linear noise

$P(f) = A_{white}^2 ((f)^{\alpha} + (\frac{f_{knee}}{f})^{\beta})$

In [ ]:
def advanced_noise_model(x, A_white, f_knee, alpha, beta):
    return A_white**2 * (1 + x*alpha + (f_knee/x)**beta)

In [ ]:
def nll_gauss(A, f_knee, alpha, beta, freq, ps, sigma):
    P_model = advanced_noise_model(freq, A, f_knee, alpha, beta)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha, beta):
       return nll_gauss(A, f_knee, alpha, beta, binned_freq, binned_ps, sigma=binned_ps_error)

    m = Minuit(nll_wrapper, A=0.1, f_knee=1, alpha=1, beta=1)
    m.limits['A'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)
    m.limits['beta'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        plt.plot(freq, ps, label="data")
        plt.plot(freq, advanced_noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Instrumental Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$', r'$\beta$']

# Fit for each instrumental configurations
for idata in range(len(names)):
    values, errors, chi2_dof= [], [], []
    for index in range(len(dict_data[names[idata]]['clean_data'])):
        if index == 0:
            plot = True
        else:
            plot = False
        val, err, red_chi2 = fit_minuit_ll(dict_data[names[idata]]['clean_data'][index], nbins=int(dict_data[names[idata]]['clean_data'][index].size/10), plot=plot, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(len(label), 5, figsize=(15, 10), sharey='row', sharex=True)

n_datasets = len(names)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(len(label)):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(len(label)):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
#    ax[1, idata].set_ylim(0, 10)

for idata in range(n_datasets):
    ax[-1, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - over f + Linear.png')
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=5, shared_xaxes=True, shared_yaxes=True,
    subplot_titles=names,
    vertical_spacing=0.05,   # decreased spacing for larger plots
    horizontal_spacing=0.03, # decreased spacing for larger plots
    row_heights=[0.1, 0.1, 0.1],
    column_widths=[0.1, 0.1, 0.1, 0.1, 0.1]
)

n_datasets = len(names)
n_points = len(params_values_loglike[0])

for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        
        # Add error bars
        fig.add_trace(
            go.Scatter(
                x=x,
                y=values,
                error_y=dict(type='data', array=errors, visible=True),
                mode='markers',
                marker=dict(color="black"),
                name=names[idata],
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )
        
        # Add mean line
        fig.add_trace(
            go.Scatter(
                x=[0, n_points-1],
                y=[mean_val, mean_val],
                mode='lines',
                line=dict(color='red', dash='dash'),
                name=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}',
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )

# Update layout
fig.update_layout(
    height=1000,
    width=2000,
    title_text=r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$',
    showlegend=False
)

# Update y-axis labels
for iparam in range(3):
    fig.update_yaxes(title_text=label[iparam], row=iparam+1, col=1)

# Update x-axis labels
for idata in range(n_datasets):
    fig.update_xaxes(title_text='Instrumental Index', row=3, col=idata+1)

fig.show()

In [ ]:
params_value = np.array(params_values_loglike, dtype=float)[index_data]
print(params_value.shape)

In [ ]:
unique_distances = np.unique(distance_base_antenna2)
colors = px.colors.sample_colorscale("jet", [i/(len(unique_distances)-1) for i in range(len(unique_distances))])

# Create a subplot figure with 2 rows and 1 column, sharing the x-axis.
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
)

# Loop over unique distances to add traces for both subplots.
for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna2 == dist
    # First subplot: A_white values
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 0],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i])
        ),
        row=1, col=1
    )
    # Second subplot: f_knee values; hide legend to avoid duplicates.
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 1],
            mode='markers',
            name=f'{dist:.1f}m',
            marker=dict(color=colors[i]),
            showlegend=False
        ),
        row=2, col=1
    )

# Update axis labels and overall layout.
fig.update_xaxes(title_text='Distance Antenna 1 to Antenna 2', row=2, col=1)
fig.update_yaxes(title_text=r'$A_{white}$', row=1, col=1)
fig.update_yaxes(title_text=r'$f_{knee}$', row=2, col=1)

fig.update_layout(
    title=f'Noise power spectrum parameters for different distances - {data_name} data',
    # Position the legend outside the plot on the right.
    legend=dict(
        title="Distance Base to Antenna 2",
        orientation='v',
        x=1.02,
        y=1,
        xanchor='left',
        yanchor='top'
    ),
    width=1200,
    height=800,
    margin=dict(r=150)  # Increase right margin to accommodate the legend.
)

fig.show()

In [ ]:
unique_distances = np.unique(distance_base_antenna2)
colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna2 == dist
    ax[0].scatter(distance_antenna1_antenna2[mask], params_value[mask, 0], label=f'{dist:.1f}', color=colors[i])
    ax[1].scatter(distance_antenna1_antenna2[mask], params_value[mask, 1], color=colors[i])

ax[0].set_ylabel(r'$A_{white}$')
ax[1].set_ylabel(r'$f_{knee}$')
ax[1].set_xlabel('Distance Antenna 1 to Antenna 2')

# Create a single legend outside the plot
handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 1), title="Distance Base to Antenna 2")

fig.suptitle(f'Noise power spectrum parameters for different distances - {data_name} data')
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust layout to make space for legend
plt.show()